# Implementing TF-IDF

Using sklearn, nltk (stopwords)

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [102]:
doc1 = 'they went to the park for bird watching'
doc2 = 'she went to the beach to swim'

We need to convert the text to a vector. First, let's recap the BoW approach:

In [7]:
bow1 = doc1.split(' ')
bow2 = doc2.split(' ')

In [10]:
# use Python's Union function to get unique words:
corpus_unique = set(bow1).union(set(bow2))

Create a dict of words & frequency for each doc in the corpus


In [12]:
num_words_doc1 = dict.fromkeys(corpus_unique, 0)

In [14]:
for word in bow1:
    num_words_doc1[word] += 1

In [16]:
# do the same for `doc2`
num_words_doc2 = dict.fromkeys(corpus_unique, 0)

for word in bow2:
    num_words_doc2[word] += 1

In [19]:
import pandas as pd

In [51]:
corpus_words = {}
for word in corpus_unique:
    corpus_words[word] = [num_words_doc1[word], num_words_doc2[word]]

In [70]:
pd.DataFrame.from_dict(corpus_words)

,she,park,swim,bird,beach,the,went,to,for,they,watching
0,0,1,0,1,0,1,1,1,1,1,1
1,1,0,1,0,1,1,1,2,0,0,0


We need to account for 'stop' word noise

In [71]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

### TF

In [72]:
def compute_term_freq(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [81]:
tf1 = compute_term_freq(num_words_doc1, bow1)
tf2 = compute_term_freq(num_words_doc2, bow2)

### IDF

In [74]:
import math

def compute_IDF(documents):
    N = len(documents)    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict


In [75]:
idfs = compute_IDF([num_words_doc1, num_words_doc2])

In [84]:
idfs

{'she': 0.6931471805599453,
 'park': 0.6931471805599453,
 'swim': 0.6931471805599453,
 'bird': 0.6931471805599453,
 'beach': 0.6931471805599453,
 'the': 0.0,
 'went': 0.0,
 'to': 0.0,
 'for': 0.6931471805599453,
 'they': 0.6931471805599453,
 'watching': 0.6931471805599453}

### TF-IDF

In [85]:
def compute_TFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [86]:
tfidf1 = compute_TFIDF(tf1, idfs)
tfidf2 = compute_TFIDF(tf2, idfs)
df = pd.DataFrame([tfidf1, tfidf2])

In [87]:
df

,she,park,swim,bird,beach,the,went,to,for,they,watching
0,0.000000,0.086643,0.000000,0.086643,0.000000,0.0,0.0,0.0,0.086643,0.086643,0.086643
1,0.099021,0.000000,0.099021,0.000000,0.099021,0.0,0.0,0.0,0.000000,0.000000,0.000000


### Direct implementation with sklearn

In [79]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([doc1, doc2])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [80]:
df

,beach,bird,for,park,she,swim,the,they,to,watching,went
0,0.00000,0.391668,0.391668,0.391668,0.00000,0.00000,0.278675,0.391668,0.278675,0.391668,0.278675
1,0.40698,0.000000,0.000000,0.000000,0.40698,0.40698,0.289569,0.000000,0.579139,0.000000,0.289569


The small difference in values is due to the use of a smoothed version of idf + other optimizations. With a larger doc, the score for `the` would be greatly reduced. [Ref](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76)
